In [66]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
sb.set_context("paper")
sb.set(rc={'figure.figsize': (10, 6)})
sb.set_style("whitegrid")
import ipywidgets as widgets

In [2]:
def N(K,b,c,p,r,𝜃max,tmax=None,T=0,𝜇=None,𝜎=None):
    if 𝜇 is None: 𝜇 = tmax / 2  # duration of a pandemic mean, 𝜇 = (𝑎+𝑏)/2
    if 𝜎 is None: 𝜎 = tmax / math.sqrt(12)  # duration of a pandemic std deviation, 𝜎 = sqrt( (𝑏−𝑎)^2 / 12) 
    return (2*K*𝜃max+2*(r-p)*𝜇*𝑇) / (2*𝑇*𝜃max+(𝜎**2+𝜇**2)) + (c-b)

# Objective function for the supplier's expected profit over the planning horizon
def Π(x,A,K,b,c,e,p,r,𝜃max,tmax,T):
    𝜇 = tmax / 2  # duration of a pandemic mean, 𝜇 = (𝑎+𝑏)/2
    𝜎 = tmax / math.sqrt(12)  # duration of a pandemic std deviation, 𝜎 = sqrt( (𝑏−𝑎)^2 / 12) 
    return (-K + (b-c+e)*T + (p-c+e-r+b)*𝜇*T/𝜃max) * x + A * ((p-b)*T + (r-b)*𝜇*T/𝜃max - (p-b)*(T*𝜇-(𝜎**2+𝜇**2)/2)/𝜃max)

# Objective function for the policy-maker's expected cost for supplying the necessary EMPs over the planning horizon
def Ω_(e,K,A,L,S,T,b,c,r,p,u,h,𝜃max,tmax):
    N_val = N(K,b,c,p,r,𝜃max,tmax,T)
    if e <= N_val:
        return (T*r)/(2*u*𝜃max*tmax) * 𝑆**2 - ( (h*T**2 + 2*T*r)/(2*𝜃max) - p - h*T  ) * 𝑆 + (u*r*T*tmax)/(2*𝜃max)
    else: # e > N
        if (u-A)*L <= S <= (u-A)*tmax:
            return (T*(u*r-p*A)/(2*𝜃max*tmax*u*(u-A))) * S**2 + (p+h*T-h*T**2/(2*𝜃max)-r*T/𝜃max) * 𝑆 + \
                    N_val*A*T + N_val*A*tmax**2/(6*𝜃max) + T*tmax/(2*𝜃max)*(p*A+r*(u-A))
        elif S >= (u-A)*tmax:
            return ((p*T)/(2*u*𝜃max*tmax)) * S**2 + (p+h*T-h*T**2/(2*𝜃max)-p*T/𝜃max) * 𝑆 + \
                    N_val*A*T + N_val*A*tmax**2/(6*𝜃max) + p*T*u*tmax/(2*𝜃max)
        else:
            return np.nan
#             raise ValueError(f'S = {S} value is out of bound')

Ω = np.vectorize(Ω_)

def Lmax(r,p,h,𝜃max,tmax,T):
    return (r - p*𝜃max/T + h*T/2 - h*𝜃max) * tmax/r

In [84]:
zeta = 0.01

fig = plt.figure()
ax = fig.gca()
sz = 100  # graph resolution

gap_x = 6e6*.005
loc_y = 1.47e8

line1, = plt.plot([0,1],[0,1],label='$\Omega(S),x^*=0,e^*=0$',color='b')
line2, = plt.plot([1,2],[1,0],label='$\Omega(S),x^*=A,e^*=N$',color='chocolate')
text3 = plt.text(0,loc_y, '$uL$',rotation=90,color='b',va='top')
line3 = plt.axvline(0,linestyle='--',color='b')
text4 = plt.text(0,loc_y, '$ut_{max}$',rotation=90,color='k',va='top')
line4 = plt.axvline(0,linestyle='--',color='k')
text5 = plt.text(0,loc_y, '$(u-A)L$',rotation=90,color='chocolate',va='top')
line5 = plt.axvline(0,linestyle='--',color='chocolate')
text6 = plt.text(0,loc_y, '$(u-A)t_{max}$',rotation=90,color='chocolate',va='top')
line6 = plt.axvline(0,linestyle='--',color='chocolate')

@widgets.interact(K = (0.0,2.0,0.05), b = (1,10), c = (1,10), p = (1,20), r = (1,100), h = (0.0,10.0,0.1), 
                  T = (3*365,15*365), L = (0,3*365), 
                  u = (2500,7500), A = (1000,7000),
                  𝜃max = (10*365,100*365), tmax = (.5*365,5*365))
def GenPlot(K = 0.1, b = 5, c = 7, p = 10, r = 40, h = 0.5, T = 5*365, L = 60, u = 5720, A = 2860, 𝜃max = 20*365, tmax = 3*365):
#     print(f'{zeta=}\n{K=}\n{b=}\n{c=}\n{p=}\n{r=}\n{h=}\n{T=}\n{L=}\n{u=}\n{A=}\n{𝜃max=}\n{tmax=}')
    display(f'L = {L:.0f}')
    display(f'tmax = {tmax:.0f}')
    display(f'Lmax(r,p,h,𝜃max,tmax,T) = {Lmax(r,p,h,𝜃max,tmax,T):.0f}')
    display(f'r*u/(r*u-p*A) = {r*u/(r*u-p*A):.2f}')
    display(f'r*u/(r*u-p*A) * Lmax = {r*u/(r*u-p*A) * Lmax(r,p,h,𝜃max,tmax,T):.0f}')

    h /= 100
    S1 = np.linspace(u*L, u*tmax, sz)
    e1 = 0
    cost1 = Ω(e1,K,A,L,S1,T,b,c,r,p,u,h,𝜃max,tmax)
    line1.set_xdata(S1)
    line1.set_ydata(cost1)

    S2 = np.linspace((u-A)*L, u*tmax, sz)
    e2 = N(K,b,c,p,r,𝜃max,tmax,T) + zeta
    cost2 = Ω(e2,K,A,L,S2,T,b,c,r,p,u,h,𝜃max,tmax)
    line2.set_xdata(S2)
    line2.set_ydata(cost2)

    val = u*L
    text3.set_position((val+gap_x,loc_y))
    line3.set_xdata(len(line3.get_xdata())*[val])
    val = u*tmax
    text4.set_position((val+gap_x,loc_y))
    line4.set_xdata(len(line4.get_xdata())*[val])
    val = (u-A)*L
    text5.set_position((val+gap_x,loc_y))
    line5.set_xdata(len(line5.get_xdata())*[val])
    val = (u-A)*tmax
    text6.set_position((val+gap_x,loc_y))
    line6.set_xdata(len(line6.get_xdata())*[val])

    ax.set_xlabel('S')
    ax.set_ylabel('Cost')
    plt.legend()
    plt.xlim(0, np.max([S1,S2])*1.1)
    plt.ylim(np.min([cost1,cost2])*.7, np.max([cost1,cost2])*1.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='K', max=2.0, step=0.05), IntSlider(value=5, descript…